<center><h1>Predict Online News Popularity</h1></center>
<center><h2>The News Squad</h2></center>
<center><h4>📸Jingwen Yu, Zhentao Hou, Xiao Chu<center><h4>

<center><h2>Outline📝</h2></center>

1. Ask - Why we care?  
2. Acquire - Introduce the dataset  
3. Process - EDA and feature engineering  
4. Model - Choose the model based on the evaluation metric  
5. Deliver - Takeaway  

<center><h2>🧐Ask - Why we care about news popularity</h2></center>
<br></br>
  
News is an important channel for us to learn what is happening in the world.
\
\
A piece of popular news could lead to a successful advertising or public relation activity, contributing great business value to companies.

 <center><h2>📩Acquire - Introduce the Dataset</h2></center>
 
1. Number of Features: 58
2. Target Column: Number of Shares
3. Number of Instances: 39644
4. Source: UCI Machine Learning Repository

<center><h2>📊Process - EDA and Feature Engineering</h2></center>

<center><h2>Target Exploration -- Raw Dataset</h2></center>

#### Target Range from 1 to 843300

![pic14.png](picture/pic14.png)

### Feature Exploration
![Picture1.png](picture/Picture1.png)

<center><h2>💻Model - Evaluation Metric and Model Selection</h2></center>
<br></br>

Regression Model
- Predict Number of Shares ➡️ **Median Absolute Error (MedAE)**

1. Create Pipelines
    - scaler - StandardScaler()
    - regressor - Lasso, Ridge, and Random Forest Regressor
2. Fit models
3. Evaluation

![Picture2.png](picture/Picture2.png)

#### Summary
- 3 regression algorithms, MedAE as the North Star metric
- Large MedAE (> 1600)
- Classification might make more sense

### Classification Model
<br></br>
- Binary Classification: Predict Popular or Unpopular ➡️ **F1 Score**

1. Choosing the threshold -- **Mean (3395) or Median (1400)**
![pic12.png](picture/pic12.png)

### Redo the EDA Again 😹
### Weekend or Weekday?
![pic17.png](picture/pic17.png)

### Which Channel?
![pic15.png](picture/pic15.png)

2. Build Pipelines
    - scaler - StandardScaler()
    - classifier - \
      LogisticRegressionCV\
      KNeighborsClassifier\
      GaussianNB\
      RandomForestClassifier

3. Fit Models & Model Evaluation

    - F1-score = $\frac{2*precision*recall}{precision+recall}$
    - F1-score is used when False Negatives and False Positives are crucial
    - F1-score is a better metric when there are imbalanced classes

![WechatIMG71.png](picture/WechatIMG71.png)

![metrics.png](picture/metrics.png)


![pic16.png](picture/pic16.png)

<center><h2>✍🏻Deliver - Takeaway</h2></center>

- Using F1-score as the North Star Metric, Random Forest Classifier is the best model.
- Recommendations for reporters and business entities:\
(1) Keywords are important.\
(2) Publication time matters.\
(3) Reference articles with high popularity would help.